In [24]:
import os 
from dotenv import load_dotenv,find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [26]:
from langchain_community.utilities import SQLDatabase

sqlite_db_path = "data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

In [27]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm,db)

response = chain.invoke({"question":"How many species of trees are in San Francisco?"})

response

'SELECT COUNT(DISTINCT "qSpecies") AS TotalSpecies\nFROM street_trees;'

In [28]:
db.run(response)

'[(148,)]'

In [29]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### Question - to - Answer (WithOut Sql in between)

In [30]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm,db)

execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question":"List the species of trees that are present in San Francisco"})

'[("Arbutus \'Marina\' :: Hybrid Strawberry Tree",), (\'Afrocarpus gracilior :: Fern Pine\',), ("Thuja occidentalis \'Emerald\' :: Emerald Arborvitae",), ("Magnolia grandiflora \'Little Gem\' :: Little Gem Magnolia",), (\'Platanus x hispanica :: Sycamore: London Plane\',), (\'Ulmus parvifolia :: Chinese Elm\',), (\'Leptospermum laevigatum :: Australian Tea Tree\',), ("Olea europaea \'Fruitless\' :: Fruitless Olive",), (\'Ficus rubiginosa :: Port Jackson Fig\',), (\'Cycas revoluta :: Sago palm\',), (\'Lophostemon confertus :: Brisbane Box\',), (\'Podocarpus gracilor :: Fern Pine\',), (\'Eriobotrya deflexa :: Bronze Loquat\',), (\'Ligustrum lucidum :: Glossy Privet\',), (\'Syagrus romanzoffianum :: Queen Palm\',), ("Ficus microcarpa nitida \'Green Gem\' :: Indian Laurel Fig Tree \'Green Gem\'",), (\'Prunus cerasifera :: Cherry Plum\',), (\':: To Be Determine\',), (\'Myoporum laetum :: Myoporum\',), (\'Tristaniopsis laurina :: Swamp Myrtle\',), (\'Ginkgo biloba :: Maidenhair Tree\',), (\'

### Question - to - LLMAnswer 

In [32]:
from operator import itemgetter 

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

sqlite_db_path = "data\street_tree_db.sqlite"

db  = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

answer_prompt = PromptTemplate.from_template(
    """ 
    Given the following user question,
    corresponding SQL query, and SQL result,
    answer the user question.

    Question : {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer:
    """
)

write_query = create_sql_query_chain(llm,db)

execute_query = QuerySQLDataBaseTool(db=db)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result = itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question":"List the species of trees that are present in San Francisco"})

"The species of trees present in San Francisco include:\n- Arbutus 'Marina' :: Hybrid Strawberry Tree\n- Afrocarpus gracilior :: Fern Pine\n- Thuja occidentalis 'Emerald' :: Emerald Arborvitae\n- Magnolia grandiflora 'Little Gem' :: Little Gem Magnolia\n- Platanus x hispanica :: Sycamore: London Plane\n- Ulmus parvifolia :: Chinese Elm\n- Leptospermum laevigatum :: Australian Tea Tree\n- Olea europaea 'Fruitless' :: Fruitless Olive\n- Ficus rubiginosa :: Port Jackson Fig\n- Cycas revoluta :: Sago palm\n- Lophostemon confertus :: Brisbane Box\n- Podocarpus gracilor :: Fern Pine\n- Eriobotrya deflexa :: Bronze Loquat\n- Ligustrum lucidum :: Glossy Privet\n- Syagrus romanzoffianum :: Queen Palm\n- Ficus microcarpa nitida 'Green Gem' :: Indian Laurel Fig Tree 'Green Gem'\n- Prunus cerasifera :: Cherry Plum\n- Myoporum laetum :: Myoporum\n- Tristaniopsis laurina :: Swamp Myrtle\n- Ginkgo biloba :: Maidenhair Tree\n- Pyrus calleryana :: Ornamental Pear\n- Morus alba 'Fruitless' :: Fruitless 